In [14]:
from src.model.system import System
from src.data.datamodule import MyDatamodule
import torch
from tqdm.notebook import tqdm
import numpy as np
from src.experiment_config import read_experiment_config
from src.model.utils import top_k_accuracy

In [43]:
system = System.load_from_checkpoint('/mnt/IMAGING/HUB/MODELS/CHECKPOINTS/cvppa/cvppa/wr_baseline_1/cp_epoch=108_val_loss=0.613_val_acc=0.812.ckpt')
system.freeze()
hparams = system.hparams
cfg = read_experiment_config('config1.yaml')

In [10]:
cfg.labels_path

'/home/AZA/PycharmProjects/competitions/cvppa2023/dataset/WR2021/images/'

In [44]:
dm = MyDatamodule(
    images_path=cfg.images_path, 
    labels_path=cfg.labels_path, 
    fold=0, 
    size=hparams.size, 
    augmentation_p=False, 
    train_batch_size=4,
    val_batch_size=8, 
    num_workers=8)
dm.setup('fit')
ds = dm.train_dataset + dm.val_dataset

In [45]:
probs = []
labels = []
names = ds.datasets[0].labels['names'].tolist() + ds.datasets[1].labels['names'].tolist() 
with torch.no_grad():
    for i in tqdm(range(len(ds))):
        img, label = ds[i]
        pred = system(img.unsqueeze(0).cuda())
        labels.append(label)
        probs.append(torch.nn.functional.softmax(system(img.unsqueeze(0).cuda())[0], dim=0).cpu().numpy())

  0%|          | 0/1332 [00:00<?, ?it/s]

In [46]:
acc = top_k_accuracy(torch.tensor(probs), torch.tensor(labels))

In [47]:
acc

[tensor([0.2282])]

[array([0.09396961, 0.0778559 , 0.1639748 , 0.01968632, 0.2966825 ,
        0.000613  , 0.34721783], dtype=float32),
 array([0.33968508, 0.14096013, 0.2162115 , 0.14112718, 0.10069911,
        0.04946569, 0.01185131], dtype=float32),
 array([0.13775875, 0.18359202, 0.04742593, 0.23069495, 0.06009063,
        0.25490454, 0.08553313], dtype=float32),
 array([2.8450281e-04, 3.6282980e-04, 1.7859747e-04, 3.0352248e-04,
        9.9849796e-01, 2.4192067e-04, 1.3064503e-04], dtype=float32),
 array([0.12405635, 0.1920402 , 0.05248982, 0.23923802, 0.044837  ,
        0.27997762, 0.06736097], dtype=float32),
 array([0.12474776, 0.11225634, 0.02308438, 0.14309433, 0.26452264,
        0.2444039 , 0.08789062], dtype=float32),
 array([0.43239298, 0.16137883, 0.12253834, 0.07172698, 0.05592104,
        0.1391704 , 0.01687143], dtype=float32),
 array([0.03846597, 0.0300674 , 0.00619181, 0.03715313, 0.74054265,
        0.04141644, 0.10616253], dtype=float32),
 array([0.07420985, 0.08850434, 0.02503789,